In [1]:
import pandas as pd
from xgboost import XGBClassifier
from xgboost import plot_importance
import matplotlib.pyplot as plt
import numpy as np

/Users/benj/miniforge3/envs/env_tf/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
base = pd.read_csv("meps_base_data.csv")
print(base)
print(base.describe())
for col in base.columns:
    if "Diagnosed" in col:
        print(base[col].value_counts())

       Unnamed: 0        id  panel  pooledWeight  age     sex      race  \
0               1  10007101     15   3603.881236   28    Male     White   
1               2  10007102     15   2544.550424   25  Female     White   
2               3  10007103     15   4050.397468    4    Male     White   
3               4  10007104     15   3064.059720    3  Female     White   
4               5  10008101     15   3635.552466   51    Male  Multiple   
...           ...       ...    ...           ...  ...     ...       ...   
61484       61485  77293101     12   5154.187341   45    Male     White   
61485       61486  77293102     12   5520.770322   41  Female     White   
61486       61487  77293103     12   5454.718112   14  Female     White   
61487       61488  77294101     12   9934.141271   54    Male     White   
61488       61489  77294102     12  11208.776977   51  Female     White   

                       married highBPDiagnosed diabetesDiagnosed  \
0                      MARRIED 

In [3]:
meds = pd.read_csv("meps_meds.csv")
print(meds)
print(meds.describe())
print(meds["rxName"].value_counts())

         Unnamed: 0        id  rxStartMonth  rxStartYear  \
0                 1  10007104             3         2011   
1                 2  10007104             3         2011   
2                 3  10008102             3         2011   
3                 4  10008102             3         2011   
4                 5  10008102             9         2011   
...             ...       ...           ...          ...   
1148342     3000952  77294102            -1           -1   
1148343     3000962  77294102            -1           -1   
1148344     3000972  77294102            -1           -1   
1148345     3000982  77294102            -1           -8   
1148346     3000992  77294102            12         2007   

                                 rxName        rxNDC  rxQuantity rxForm  
0                           AMOXICILLIN    143988775       75.00   SUSR  
1                    OTIC EDGE SOLUTION  68032032814       14.00    SOL  
2        NASAL DECONGESTANT 0.05% SPRAY  63981056903     

In [4]:
def process_base(df):
    result = df[["id", "age", "sex", "married", "highBPDiagnosed"]]
    with pd.option_context("mode.chained_assignment", None):
        result["age"] = result["age"].replace(-1, result["age"].median())
    bad_vals = [
        "UNDER 16 - INAPPLICABLE",
        "Inapplicable",
        "not ascertained",
        "Refused",
        "DK",
    ]
    for val in bad_vals:
        result = result[result != val]
    result["highBPDiagnosed"] = result["highBPDiagnosed"].replace(
        {"Yes": 1}, regex=True
    )
    result["highBPDiagnosed"] = result["highBPDiagnosed"].replace({"No": 0}, regex=True)
    result["sex"] = result["sex"].replace({"Female": 0}, regex=True)
    result["sex"] = result["sex"].replace({"Male": 1}, regex=True)
    result = result.sort_values(by=["id"])
    result = result.dropna()
    result = result.drop_duplicates()
    married_hots = pd.get_dummies(result["married"])
    result = result.drop("married", axis=1)
    result = pd.concat((result, married_hots), axis=1)
    cols = list(result.columns)
    cols.remove("highBPDiagnosed")
    cols.append("highBPDiagnosed")
    result = result[cols]
    return result


def process_meds(df, base_ids, threshold=1000):
    result = df[["id", "rxName"]]
    result = result[result["id"].isin(base_ids)]
    result = result.groupby("rxName").filter(lambda x: len(x) > threshold)
    result = result.sort_values(by=["id"])
    result = result.dropna()
    result = result.drop_duplicates()
    return result


def multihot_encode_meds(df):
    result = df.pivot_table(
        index=["id"], columns=["rxName"], aggfunc=[len], fill_value=0
    )
    result.columns = [x[1] for x in result.columns]
    return result

In [5]:
base = pd.read_csv("meps_base_data.csv")
meds = pd.read_csv("meps_meds.csv")

base_df = process_base(base)
base_ids = set(base_df["id"])

meds_df = process_meds(meds, base_ids)
meds_ids = set(meds_df["id"])

base_df = base_df[base_df["id"].isin(meds_ids)]
base_ids = set(base_df["id"])

assert base_ids == meds_ids

vec_df = multihot_encode_meds(meds_df)
result = pd.merge(
    vec_df,
    base_df,
    how="inner",
    on="id",
)
result["age"] = (result["age"] - result["age"].min()) / (
    result["age"].max() - result["age"].min()
)
result_npy = result.to_numpy()

In [6]:
model = XGBClassifier()
model.fit(result_npy[:, 1:-1], result_npy[:, -1])
top20 = np.argsort(model.feature_importances_)[:20]
cols = list(result.columns)
for top_feature in top20:
    print(cols[top_feature])
# In the top 20 important features, these antihypertensive and hyperlipidemia meds appear:
#     Clonidine, Benicar, DOXAZOSIN, Simvastatin, Crestor
# Interestingly, separation appears to correlate with high blood pressure.

/Users/benj/miniforge3/envs/env_tf/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:12:38] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
WARFARIN SODIUM
CLONIDINE
SINGULAIR (UNIT OF USE)
RANITIDINE
FERROUS SULFATE
BENICAR HCT
DIVORCED IN ROUND
DOXAZOSIN MESYLATE
PANTOPRAZOLE
KLOR-CON M20
FLUOXETINE
SINGULAIR
CITALOPRAM HYDROBROMIDE
SIMVASTATIN (FILM-COATED)
FLUTICASONE PROPIONATE
CRESTOR
SEPARATED IN ROUND
ZOLPIDEM
SERTRALINE HYDROCHLORIDE
sex
